In [243]:
import json
import os

files = os.listdir()

print(files)

['gpt-3.5-turbo_0.7_propose1_value1_greedy2_start233_end241.json', 'gpt-3.5-turbo_0.7_propose1_value1_greedy2_start0_end1.json', 'gpt-3.5-turbo_0.7_propose1_value1_greedy2_start121_end150.json', 'gpt-3.5-turbo_0.7_propose1_value1_greedy2_start58_end100.json', 'gpt-3.5-turbo_0.7_propose1_value1_greedy2_start100_end120.json', 'Untitled.ipynb', 'gpt-3.5-turbo_0.7_propose1_value1_greedy2_start150_end200.json', 'gpt-3.5-turbo_0.7_propose1_value1_greedy2_start275_end307.json', 'gpt-3.5-turbo_0.7_propose1_value1_greedy2_start57_end58.json', 'gpt-3.5-turbo_0.7_propose1_value1_greedy2_start250_end275.json', 'gpt-3.5-turbo_0.7_propose1_value1_greedy2_start1_end56.json', 'gpt-3.5-turbo_0.7_propose1_value1_greedy2_start241_end250.json', 'gpt-3.5-turbo_1.0_sample5_value1_greedy1_start0_end307.json', '.ipynb_checkpoints', 'gpt-3.5-turbo_0.7_propose1_value1_greedy2_start200_end250.json']


In [198]:
files_to_parse = ['gpt-3.5-turbo_1.0_sample5_value1_greedy1_start0_end307.json']
all_results = []

for f in files_to_parse:
    content = json.loads(open(f).read())
    all_results += content

In [199]:
entry_template = {
    'question_index': None,
    'question': None,
    'completion_tokens': None,
    'prompt_tokens': None,
    'cost': None,
    'correct_answer': None,
    'model_answer_1': None,
    'model_reasoning_1': None,
    'model_correct_1': False,
    'model_answer_2': None,
    'model_reasoning_2': None,
    'model_correct_2': False,
    'question_solved': False
}

entries = []
for res in all_results:
    entry = entry_template.copy()
    entry['question_index'] = res['idx']
    entry['question'] = res['steps'][0]['x']
    entry['completion_tokens'] = res['usage_so_far']['completion_tokens']
    entry['prompt_tokens'] = res['usage_so_far']['prompt_tokens']
    entry['cost'] = res['usage_so_far']['cost']
    entry['correct_answer'] = res['infos'][0]['correct_answer']
    entry['model_answer_1'] = res['infos'][0]['model_answer']
    entry['model_reasoning_1'] = res['ys'][0]
    entry['model_correct_1'] = int(res['infos'][0]['r'])

    if len(res['infos']) == 2:
        entry['model_answer_2'] = res['infos'][1]['model_answer']
        entry['model_reasoning_2'] = res['ys'][1]
        entry['model_correct_2'] = int(res['infos'][1]['r'])

    else:
        entry['model_answer_2'] = None
        entry['model_reasoning_2'] = None
        entry['model_correct_2'] = 0

    
    entry['question_solved'] = int(entry['model_correct_1'] or entry['model_correct_2'])
    entries.append(entry)

In [200]:
import pandas as pd
df = pd.DataFrame.from_records(entries)

cost_update = df[df['cost'] < df['cost'].shift(1)].index
for ind in cost_update:
    df.loc[ind, ['cost', 'completion_tokens', 'prompt_tokens']] += df.loc[ind-1, ['cost', 'completion_tokens', 'prompt_tokens']]

df.sort_values(by='question_index', inplace=True)
df.tail()

,question_index,question,completion_tokens,prompt_tokens,cost,correct_answer,model_answer_1,model_reasoning_1,model_correct_1,model_answer_2,model_reasoning_2,model_correct_2,question_solved
302,302,"If $f(x)$, whose graph is shown below, is defi...",1199555,5783380,11.074180,6.0,5.0,"To find the maximum value of $f^{-1}(x)$, we c...",0,None,None,0,0
303,303,"What is the ordered pair of real numbers $(x, ...",1202103,5800592,11.105094,-18.0,-1.0,Step 1: The absolute value function is always ...,0,None,None,0,0
304,304,"A circle is centered at $(5,15)$ and has a rad...",1206909,5819116,11.142492,12.0,12.0,"Step 1: Let the $y$-coordinate be $a$, so the ...",1,None,None,0,1
305,305,What real values of $x$ are not in the domain ...,1209859,5836582,11.174591,-4.0,-5.0,Step 1: To determine the values of $x$ that ar...,0,None,None,0,0
306,306,"Two weeks ago, Alice came home from vacation a...",1211834,5853779,11.204337,228.3,852.0,Step 1: Let $h$ represent the height of the pl...,0,None,None,0,0


In [202]:
df['question_solved'].mean()

0.2736156351791531

In [203]:
df['question_index'].nunique()

307